# V1 프로젝트 - 탐색적 데이터 분석 (EDA)

이 주피터 노트북은 '중고차 가격 예측 프로젝트'의 V1 버전 데이터를 기반으로 한 탐색적 데이터 분석(EDA) 과정을 담고 있습니다.
데이터의 특징을 이해하고, 주요 피처(Feature)와 타겟(Target) 변수 간의 관계를 시각적으로 확인하며, 모델링에 필요한 인사이트를 도출하는 것이 목표입니다.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 시각화 설정
sns.set_style('whitegrid')
plt.rcParams['font.family'] = 'Malgun Gothic' # 한글 폰트 설정 (Windows 기준)
plt.rcParams['axes.unicode_minus'] = False # 마이너스 폰트 깨짐 방지

ModuleNotFoundError: No module named 'seaborn'

## 1. 데이터 로드 및 기본 정보 확인
V1 프로젝트에서 사용했던 `used_cars.csv` 데이터를 로드하고, 기본적인 정보를 확인합니다.

In [ ]:
df = pd.read_csv('../data/used_cars.csv') # 데이터 폴더 경로에 맞춰 수정

print("데이터셋 상위 5행:\n", df.head())
print("\n데이터셋 정보:\n")
df.info()
print("\n데이터셋 기술 통계량:\n", df.describe())

## 2. 데이터 전처리 (V1 기준)
V1 프로젝트에서 수행했던 주요 전처리 과정을 재현합니다.
- `price`, `milage` 열 숫자 변환
- `engine` 열에서 `horsepower`, `engine_L`, `cylinders` 추출
- `model_year`를 이용한 `car_age` 생성
- 이상치 제거 (상위 5% 가격)

In [ ]:
# 'price' 열: '$'와 ',' 제거 후 숫자(float)로 변환
df['price'] = df['price'].str.replace('$', '').str.replace(',', '').astype(float)

# 'milage' 열: ' mi.'와 ',' 제거 후 숫자(int)로 변환
df['milage'] = df['milage'].str.replace(' mi.', '').str.replace(',', '').astype(int)

# 'horsepower' 열: 'engine' 열에서 마력(HP) 정보 추출 (정규표현식 사용)
df['horsepower'] = df['engine'].str.extract(r'(\d+\.?\d*)\s*HP').astype(float)

# 'engine_L' 열: 'L' 앞에 있는 숫자 추출
df['engine_L'] = df['engine'].str.extract(r'(\d+\.?\d*)\s*L').astype(float)

# 'cylinders' 열: 'Cylinder' 또는 'V' 앞에 있는 숫자 추출
df['cylinders'] = df['engine'].str.extract(r'(\d)\s*(?:Cylinder|V)').astype(float)

# 'car_age' 피처 생성
current_year = 2025 
df['car_age'] = current_year - df['model_year']

# 불필요한 열 삭제
df.drop(['engine', 'model_year'], axis=1, inplace=True)

# V1 이상치 처리 (가격 상위 5% 제거)
price_95th = df['price'].quantile(0.95)
df = df[df['price'] < price_95th]

print("\n전처리 후 데이터셋 정보:\n")
df.info()
print("\n전처리 후 데이터셋 상위 5행:\n", df.head())

## 3. 결측치 확인 (V1 기준)
V1에서는 `dropna()`로 결측치를 처리했습니다. EDA 단계에서는 어떤 열에 결측치가 얼마나 있는지 확인합니다.

In [ ]:
print("\n결측치 개수:\n", df.isnull().sum())
print("\n결측치 비율 (%):\n", df.isnull().sum() / len(df) * 100)

## 4. 주요 변수 분포 탐색
타겟 변수인 `price`와 주요 수치형 피처들의 분포를 시각화하여 데이터의 특성을 파악합니다.

In [ ]:
# price 분포
plt.figure(figsize=(10, 6))
sns.histplot(df['price'], bins=50, kde=True)
plt.title('차량 가격 분포')
plt.xlabel('가격 ($)')
plt.ylabel('빈도')
plt.show()

# milage 분포
plt.figure(figsize=(10, 6))
sns.histplot(df['milage'], bins=50, kde=True)
plt.title('주행거리 분포')
plt.xlabel('주행거리 (mi.)')
plt.ylabel('빈도')
plt.show()

# horsepower 분포
plt.figure(figsize=(10, 6))
sns.histplot(df['horsepower'].dropna(), bins=50, kde=True) # 결측치 제외
plt.title('마력 분포')
plt.xlabel('마력 (HP)')
plt.ylabel('빈도')
plt.show()

# car_age 분포
plt.figure(figsize=(10, 6))
sns.histplot(df['car_age'], bins=20, kde=True)
plt.title('차량 연식 분포')
plt.xlabel('차량 연식 (년)')
plt.ylabel('빈도')
plt.show()